In [1]:
import pandas as pd

In [4]:
df = pd.read_csv('model_ready.csv')

In [5]:
df.head()

,id,amenities,total_price,price_log,9176,0,10019,10023,139 S,37738,...,Shared room,Airbed,Couch,Futon,Pull-out Sofa,Real Bed,accommodates,bathrooms,bedrooms,beds
0,20862235,"{TV,Wifi,Kitchen,""Pets allowed"",""Free street p...",225.0,5.416100,0,0,0,0,0,0,...,0,0,0,0,0,1,-0.631105,-0.475643,-1.301959,0.026511
1,26165453,"{TV,""Cable TV"",Internet,Wifi,Kitchen,""Free par...",270.0,5.598422,0,0,0,0,0,0,...,0,0,0,0,0,1,-0.631105,-0.475643,-0.382544,-0.584999
2,23371066,"{TV,Wifi,Kitchen,""Free street parking"",""Indoor...",120.0,4.787492,0,0,0,0,0,0,...,0,0,0,0,0,1,-0.631105,-0.475643,-0.382544,-0.584999
3,29922373,"{TV,Wifi,""Air conditioning"",Pool,Kitchen,""Free...",5000.0,8.517193,0,0,0,0,0,0,...,0,0,0,0,0,1,1.650944,4.883494,3.295118,1.861040
4,20541717,"{TV,Wifi,""Air conditioning"",Kitchen,""Free park...",185.0,5.220356,0,0,0,0,0,0,...,0,0,0,0,0,1,-0.631105,-0.475643,-0.382544,-1.196509


In [6]:
df = df.drop(['amenities', 'id'], axis=1)

In [56]:
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
from sklearn import linear_model
import sklearn.metrics as metrics
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression as Lin_Reg
from sklearn.linear_model import RidgeCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
import scipy.stats as stats
import math
%matplotlib inline

In [27]:
# Split data into X and y

X = df.drop(['total_price', 'price_log'], axis=1)
y_log = df['price_log']
y = df['total_price']

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y_log, test_size=0.35)

In [48]:
reg_params = 10.**np.linspace(-10, 5, 10)
model = RidgeCV(alphas=reg_params, fit_intercept=True, cv=5)

In [49]:
model.fit(X_train, y_train)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number6.870460e-19
  overwrite_a=True).T
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number7.907628e-19
  overwrite_a=True).T
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number1.153704e-17
  overwrite_a=True).T
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guara

RidgeCV(alphas=array([1.00000e-100, 1.14976e-099, ..., 8.69749e+003, 1.00000e+005]),
    cv=5, fit_intercept=True, gcv_mode=None, normalize=False, scoring=None,
    store_cv_values=False)

In [50]:
y_pred = model.predict(X_test)

In [51]:
r2 = r2_score(y_test, y_pred)
rmse = (np.sqrt(mean_squared_error(y_test, y_pred)))

In [52]:
r2, rmse

(0.7435878910354498, 0.43463198628706384)

In [59]:
y_pred = pd.DataFrame(data=y_pred.flatten())

In [61]:
y_pred = y_pred[0].apply(lambda x: math.exp(x))

In [62]:
y_test = y_test.apply(lambda x: math.exp(x))

In [69]:
y_test = pd.DataFrame(y_test)
y_test = y_test.reset_index()
y_test = y_test.drop('index', axis=1)

In [72]:
y_pred = pd.DataFrame(y_pred)

In [75]:
pred_df = pd.concat((y_test, y_pred), axis=1)

In [78]:
# WARNING -- Do Not Run outside of Sagemaker

In [77]:
import statsmodels.api as sm
# Create OLS model with robust standard errors.
model = sm.OLS(y_log, X)
results = model.fit(cov_type='HC3')
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:              price_log   R-squared:                       0.748
Model:                            OLS   Adj. R-squared:                  0.746
Method:                 Least Squares   F-statistic:                       nan
Date:                Tue, 25 Jun 2019   Prob (F-statistic):                nan
Time:                        03:41:19   Log-Likelihood:                -25340.
No. Observations:               43365   AIC:                         5.138e+04
Df Residuals:                   43015   BIC:                         5.442e+04
Df Model:                         349                                         
Covariance Type:                  HC3                                         
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
 9176                     -0